In [142]:
import pandas as pd
import numpy as np
from finance_byu.summarize import summary

In [143]:
df = pd.read_parquet("~/Data/BOQ_data.parquet")

df.tail()

,A2ME,BEME,BEME_adj,BEME_ind,CEI,DATE,DEC_ME,DEC_SHROUT,Div,Div_ann,...,s2invt,s2rect,sale,sale_ind,sales_g,sga2s,spread_mean,std_turn,std_vol,suv
3568650,0.211263,0.114912,-0.746238,0.861151,-12.189408,2023-08-31,3.897415e+08,3164000.0,0.0,0.0,...,6.344887,26.084534,81462.0,579615.973,0.513517,0.086187,0.000152,0.708916,2.250092e+07,-1.580970
3568651,0.211263,0.114912,-0.750723,0.865635,-11.613183,2023-09-30,3.897415e+08,3164000.0,0.0,0.0,...,6.344887,26.084534,81462.0,579615.973,0.513517,0.086187,0.000133,0.465122,1.476294e+07,-0.371920
3568652,0.211263,0.114912,-0.731231,0.846143,-12.752123,2023-10-31,3.897415e+08,3164000.0,0.0,0.0,...,6.344887,26.084534,81462.0,579615.973,0.513517,0.086187,0.000177,0.531342,1.686876e+07,0.809301
3568653,0.211263,0.114912,-0.731231,0.846143,-7.943286,2023-11-30,3.897415e+08,3164000.0,0.0,0.0,...,6.344887,26.084534,81462.0,579615.973,0.513517,0.086187,0.000245,0.614813,1.954419e+07,0.278301
3568654,0.211263,0.114912,-0.731231,0.846143,-9.359996,2023-12-31,3.897415e+08,3164000.0,0.0,0.0,...,6.344887,26.084534,81462.0,579615.973,0.513517,0.086187,0.000186,0.599491,1.905735e+07,0.967711


In [144]:
# For this strategy we only need beme (book to market ratio) and the typical variables (permno, date, prc, ret)
keep = ['DATE', 'PERMNO', 'BEME', 'BEME_adj', 'PRC', 'RET']
df = df[keep]

# I like my variable lowercase and snake_case
df = df.rename(columns={'DATE':'caldt','PERMNO':'permno','BEME':'beme', 'BEME_adj':'beme_adj', 'PRC':'prc', 'RET': 'ret'})

# It's good practice to sort your data by permno and date in case it hasn't been sorted already
df = df.sort_values(['permno','caldt'])

# Later I will use month date to bin since it's possible (but highly unlikely) that different stocks have different month ending dates
df['mdt'] = df['caldt'].dt.strftime("%Y-%m")

# It's important to reset the index after cleaning
df = df.reset_index(drop=True)

# You can use this line here to play with how far back the data goes
# df = df[df['caldt'] >= '2000-01-01']

df.tail()

,caldt,permno,beme,beme_adj,prc,ret,mdt
3534200,2023-08-31,93436.0,0.114912,-0.746238,258.079987,-0.034962,2023-08
3534201,2023-09-30,93436.0,0.114912,-0.750723,250.220001,-0.030456,2023-09
3534202,2023-10-31,93436.0,0.114912,-0.731231,200.839996,-0.197346,2023-10
3534203,2023-11-30,93436.0,0.114912,-0.731231,240.080002,0.195379,2023-11
3534204,2023-12-31,93436.0,0.114912,-0.731231,248.479996,0.034988,2023-12


In [145]:
# Log returns have this cool property that they can be added together without worrying about compounding
df['log_ret'] = np.log(1 + df['ret'])

# This is how long we plan on holding each portfolio for 
holding_period = 1

# We take the rolling sum for our holding period and shift it to account for any look ahead bias
df[f'ret_{holding_period}'] = df.groupby('permno').rolling(holding_period,holding_period)['log_ret'].sum().reset_index(drop=True)
df[f'ret_{holding_period}'] = df.groupby('permno')[f'ret_{holding_period}'].shift(-(holding_period-1))

In [146]:
# We need to lag any variables that we use to form our portfolios
df['beme_lag'] = df.groupby('permno')['beme'].shift(1)

# It's common practice to filter out low priced stocks (less than $5 per share)
df['prc_lag'] = df.groupby('permno')['prc'].shift(1)

# We also filter out any rows that don't have our binning variable
df = df.query("beme_lag == beme_lag and prc_lag >= 5").reset_index(drop=True)

df.tail()

,caldt,permno,beme,beme_adj,prc,ret,mdt,log_ret,ret_1,beme_lag,prc_lag
2071643,2023-08-31,93436.0,0.114912,-0.746238,258.079987,-0.034962,2023-08,-0.035588,-0.035588,0.114912,267.429993
2071644,2023-09-30,93436.0,0.114912,-0.750723,250.220001,-0.030456,2023-09,-0.030929,-0.030929,0.114912,258.079987
2071645,2023-10-31,93436.0,0.114912,-0.731231,200.839996,-0.197346,2023-10,-0.219832,-0.219832,0.114912,250.220001
2071646,2023-11-30,93436.0,0.114912,-0.731231,240.080002,0.195379,2023-11,0.178464,0.178464,0.114912,200.839996
2071647,2023-12-31,93436.0,0.114912,-0.731231,248.479996,0.034988,2023-12,0.034390,0.034390,0.114912,240.080002


In [147]:
# This is where the magic happens
# We group the data on the month level and then sort the rows into 10 equally large bins based on book to market value
# 0 is low beme while 9 is high bme
df['beme_bins'] = df.groupby("mdt")['beme_lag'].transform(lambda x: pd.qcut(x, 10, labels=False))

df.tail()

,caldt,permno,beme,beme_adj,prc,ret,mdt,log_ret,ret_1,beme_lag,prc_lag,beme_bins
2071643,2023-08-31,93436.0,0.114912,-0.746238,258.079987,-0.034962,2023-08,-0.035588,-0.035588,0.114912,267.429993,0
2071644,2023-09-30,93436.0,0.114912,-0.750723,250.220001,-0.030456,2023-09,-0.030929,-0.030929,0.114912,258.079987,0
2071645,2023-10-31,93436.0,0.114912,-0.731231,200.839996,-0.197346,2023-10,-0.219832,-0.219832,0.114912,250.220001,0
2071646,2023-11-30,93436.0,0.114912,-0.731231,240.080002,0.195379,2023-11,0.178464,0.178464,0.114912,200.839996,0
2071647,2023-12-31,93436.0,0.114912,-0.731231,248.479996,0.034988,2023-12,0.034390,0.034390,0.114912,240.080002,0


In [148]:
# We then create a portfolio dataframe that consists of 10 equally weighted beme portfolios
# Again, portfolio 0 is low beme while portfolio 9 is high beme
port = df.groupby(['caldt', 'beme_bins'])[f'ret_{holding_period}'].mean().unstack(level=['beme_bins'])*100

# Drop rows from port where we don't know the holding return (in this case the most recent month)
port = port.dropna()

# We create a long short portfolio where we long the high beme portfolio and short the low beme portfolio
# This portfolio is called zero-cost since it has not initial outlay
# It will also be closer to market neutral (beta of 0) and have higher returns than the other portfolios on average
port['spread'] = port[9] - port[0]

port.tail()

beme_bins,0,1,2,3,4,5,6,7,8,9,spread
caldt,,,,,,,,,,,
2023-08-31,-6.132560,-5.474655,-5.859880,-5.101899,-5.023423,-7.031734,-5.269149,-6.327478,-4.268978,-6.150884,-0.018324
2023-09-30,-7.365525,-6.750535,-7.695025,-6.179272,-6.495925,-6.436997,-6.334325,-5.859223,-5.156596,-7.089162,0.276363
2023-10-31,-8.134711,-8.462988,-8.720953,-8.177769,-6.873607,-6.253841,-5.009432,-5.258696,-7.822518,-7.905047,0.229664
2023-11-30,10.449136,8.669060,8.066940,7.618520,7.626173,8.317522,8.718810,7.929401,7.037371,7.639633,-2.809504
2023-12-31,8.916030,8.529981,10.835894,9.472151,8.869084,11.067365,11.136093,11.066214,12.923121,9.945568,1.029538


In [149]:
# This is a nice function that some BYU quant professors made to look at the performance of portfolios
# The mean value here would be the average 1 month return for each portfolio over the history of our dataset
# The other important value is the tstat. Generally a tstat of 2 or higher (equivalent to p-value < .05) is statistically significant.
# We see that all of the portfolios are statistically significant and that our spread portfolio has a high return (1% per month ~ 12% annually)
# and is also statistically significant! Note that if our portfolio has a beta of 0 then this would be 9.6% alpha which is super high (so something might be fishy)
summary(port).loc[['count','mean','std','tstat']].round(3)

beme_bins,0,1,2,3,4,5,6,7,8,9,spread
count,809.000,809.000,809.000,809.000,809.000,809.000,809.000,809.000,809.000,809.000,809.000
mean,-0.425,-0.074,0.185,0.318,0.421,0.537,0.599,0.613,0.733,0.669,1.094
std,6.978,6.235,5.717,5.536,5.348,5.149,4.993,5.036,4.960,5.499,4.390
tstat,-1.734,-0.336,0.919,1.635,2.238,2.965,3.411,3.461,4.206,3.459,7.089
